In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('Input/Data.xlsx')
df.head(3)

,\rho(g/cm^3),T_c(K),T_f(K),c_b(ppm),3S_{tr1}(1),3S_{tr2}(2),\Sigma_{a1}(3),\Sigma_{a2}(4),nuS_{f1}(5),nuS_{f2}(6),\Sigma_{f1}(7),\Sigma_{f2}(8),\Sigma_{1->2}(9),k_{inf}(10),\sigma_{a2}^{Xe},\sigma_{a2}^{Sm}
0,0.7235,575.0,1027.0,600.0,0.661853,2.568298,0.009376,0.085584,0.006677,0.133250,0.002620,0.054750,0.017392,1.261041,1072970.0,35421.398
1,0.7750,448.0,1043.0,1500.0,0.679602,2.874687,0.009581,0.098339,0.006723,0.139256,0.002636,0.057218,0.018726,1.174265,1141000.0,36147.801
2,0.9125,370.5,1418.0,750.0,0.727336,3.668742,0.009908,0.102815,0.006844,0.153823,0.002681,0.063203,0.022463,1.249619,1308480.0,39214.000


In [3]:
features = ['\\rho(g/cm^3)',
            'T_c(K)',
           'c_b(ppm)']

In [4]:
df[features]

,\rho(g/cm^3),T_c(K),c_b(ppm)
0,0.723500,575.00000,600.00000
1,0.775000,448.00000,1500.00000
2,0.912500,370.50000,750.00000
3,0.637500,525.50000,2250.00000
4,0.706250,409.25000,375.00000
...,...,...,...
252,0.639648,369.28909,972.65619
253,0.914648,524.28912,2472.65600
254,0.777148,446.78909,222.65620
255,0.502148,601.78912,1722.65600


In [5]:
from sklearn.preprocessing import StandardScaler

pca = StandardScaler()
pca.fit(df[features])
# Выход pca - numpy матрица, положим ее в новую переменную со всеми фичами
X = pca.transform(df[features])

In [6]:
# Не будем сжимать пространство регрессоров
X[:,0] = X[:,0]/1
X[:,1] = X[:,1]/1
X[:,2] = X[:,2]/1

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, df['\Sigma_{a2}(4)'], test_size=0.2)

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score, mean_squared_error

In [10]:
def print_metrics(y_preds, y):
    print(f'R^2: {r2_score(y_preds, y)}')
    print(f'MSE: {mean_squared_error(y_preds, y)}')

In [11]:
from sklearn.metrics import r2_score, mean_squared_error, make_scorer
from sklearn.model_selection import cross_validate

In [12]:
cross_validate(KNeighborsRegressor(n_neighbors = 8, weights = 'distance'), X, df['\Sigma_{a2}(4)'], cv=5,
               scoring={'mean_squared_error': make_scorer(mean_squared_error)})

{'fit_time': array([0.0009973 , 0.00099826, 0.00099707, 0.        , 0.00099707]),
 'score_time': array([0.00099707, 0.00099754, 0.        , 0.00102425, 0.        ]),
 'test_mean_squared_error': array([3.83939679e-06, 3.65970379e-06, 3.31815347e-06, 3.61910234e-06,
        3.05917557e-06])}

In [13]:
# Сожмем пространство признаков по значимости регрессоров
X[:,0] = X[:,0]/30
X[:,1] = X[:,1]/10
X[:,2] = X[:,2]/1

X_train, X_test, y_train, y_test = train_test_split(X, df['\Sigma_{a2}(4)'], test_size=0.2)

cross_validate(KNeighborsRegressor(n_neighbors = 8, weights = 'distance'), X, df['\Sigma_{a2}(4)'], cv=5,
               scoring={'mean_squared_error': make_scorer(mean_squared_error)})

{'fit_time': array([0.        , 0.00099707, 0.00101185, 0.        , 0.00099921]),
 'score_time': array([0.00097108, 0.00099659, 0.00098228, 0.00102544, 0.        ]),
 'test_mean_squared_error': array([6.70116497e-05, 6.55473115e-05, 8.41571670e-05, 6.98431297e-05,
        7.35042726e-05])}

In [14]:
# Разожмем пространство прзнаков
X[:,0] = X[:,0]*30*30
X[:,1] = X[:,1]*10*10
X[:,2] = X[:,2]*1*1

X_train, X_test, y_train, y_test = train_test_split(X, df['\Sigma_{a2}(4)'], test_size=0.2)

cross_validate(KNeighborsRegressor(n_neighbors = 8, weights = 'distance'), X, df['\Sigma_{a2}(4)'], cv=5,
               scoring={'mean_squared_error': make_scorer(mean_squared_error)})

{'fit_time': array([0.        , 0.00099707, 0.        , 0.        , 0.00099754]),
 'score_time': array([0.00099635, 0.00099802, 0.00099659, 0.00099373, 0.        ]),
 'test_mean_squared_error': array([3.32462285e-05, 3.46806467e-05, 2.77146588e-05, 3.69761404e-05,
        3.20303137e-05])}